In [ ]:
from google import genai
from google.genai import types
import time




In [ ]:
client = genai.Client()



In [ ]:
# ---------------------------------------------------------
# 1. Upload multiple files (these can be ANY files you want)
# ---------------------------------------------------------
file1 = client.files.upload(file='sample1.txt', config={'name': 'sample_file_1'})
file2 = client.files.upload(file='sample2.txt', config={'name': 'sample_file_2'})



In [ ]:
# ---------------------------------------------------------
# 2. Create TWO File Search Stores
# ---------------------------------------------------------
store_A = client.file_search_stores.create(
    config={'display_name': 'Store-A-Documents'}
)

store_B = client.file_search_stores.create(
    config={'display_name': 'Store-B-Documents'}
)



In [ ]:
# ---------------------------------------------------------
# 3. Import file1 into Store A
# ---------------------------------------------------------
op1 = client.file_search_stores.import_file(
    file_search_store_name=store_A.name,
    file_name=file1.name
)

# 4. Import file2 into Store B
op2 = client.file_search_stores.import_file(
    file_search_store_name=store_B.name,
    file_name=file2.name
)



In [ ]:
# ---------------------------------------------------------
# 5. Wait for both imports to complete
# ---------------------------------------------------------
while not op1.done or not op2.done:
    time.sleep(5)
    if not op1.done:
        op1 = client.operations.get(op1)
    if not op2.done:
        op2 = client.operations.get(op2)



In [ ]:
# ---------------------------------------------------------
# 6. Query BOTH vector stores at the same time
# ---------------------------------------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Give me information related to both documents.",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[
                        store_A.name,
                        store_B.name
                    ]
                )
            )
        ]
    )
)

print(response.text)

In [ ]:
print(response.text)